In [1]:
import time as t

### Pre-Configuration of Spark

function sparknotebook()
{
export SPARK_LOCAL_IP="192.168.10.1"
export PYSPARK_DRIVER_PYTHON=jupyter
export PYSPARK_DRIVER_PYTHON_OPTS="notebook --no-browser --NotebookApp.token='' --allow-root --NotebookApp.ip='*' --NotebookApp.port=8880"
nohup $SPARK_HOME/bin/pyspark --master spark://192.168.10.1:7077
}

*** NO ANY CRONTAB JOB ***

In [2]:
! rm -rf metastore_db/

In [3]:
spark.stop()

In [4]:
spark = SparkSession \
.builder \
.appName("Prediction of Loan Payment") \
.master("spark://192.168.10.1:7077") \
.config("spark.executor.cores","1")\
.getOrCreate();

In [5]:
ls -l

total 688820
-rw-r--r-- 1 root root 689413344 Dec  8  2019 2008.csv
-rw-r--r-- 1 root root     19549 Feb  8  2020 Aekanun-Collection-GKBM-NEW (Update).ipynb
-rw-r--r-- 1 root root    174778 Oct 25 14:50 BEST2020-Prediction of Loan Payment.ipynb
-rw-r--r-- 1 root root     94730 Feb  8  2020 Best-LAB-Dataproc-Prediction_of_Loan_Payment-Data2016-2019Q1-11Oct2019.ipynb
-rw-r--r-- 1 root root      3560 Feb  8  2020 Homework_Pond.ipynb
-rw-r--r-- 1 root root      4188 Feb  8  2020 Pond_(HDFS-Managing_Files).ipynb
-rw-r--r-- 1 root root     47741 Feb  8  2020 Pond_(Hive_Movielen).ipynb
-rw-r--r-- 1 root root    134422 Feb 29  2020 Pond_01-Feb-20+(Predict+ChargeOff)-Copy1.ipynb
-rw-r--r-- 1 root root    128831 Feb 25  2020 Pond_01-Feb-20+(Predict+ChargeOff).ipynb
-rw-r--r-- 1 root root     19082 Feb  8  2020 Pond_07-Sep-19 (Homework).ipynb
-rw-r--r-- 1 root root      9710 Feb  8  2020 Pond_07-Sep-19 (Time Table).ipynb
-rw-r--r-- 1 root root    115459 Dec  8  2019 Pond_08-Dec-19 (Arrival Delay)

In [6]:
! pwd

/root/notebook/POND


In [7]:
spark

In [8]:
! ls /root/notebook/data/LoanStats_web.csv

/root/notebook/data/LoanStats_web.csv


In [9]:
ls -l

total 688820
-rw-r--r-- 1 root root 689413344 Dec  8  2019 2008.csv
-rw-r--r-- 1 root root     19549 Feb  8  2020 Aekanun-Collection-GKBM-NEW (Update).ipynb
-rw-r--r-- 1 root root    174778 Oct 25 14:50 BEST2020-Prediction of Loan Payment.ipynb
-rw-r--r-- 1 root root     94730 Feb  8  2020 Best-LAB-Dataproc-Prediction_of_Loan_Payment-Data2016-2019Q1-11Oct2019.ipynb
-rw-r--r-- 1 root root      3560 Feb  8  2020 Homework_Pond.ipynb
-rw-r--r-- 1 root root      4188 Feb  8  2020 Pond_(HDFS-Managing_Files).ipynb
-rw-r--r-- 1 root root     47741 Feb  8  2020 Pond_(Hive_Movielen).ipynb
-rw-r--r-- 1 root root    134422 Feb 29  2020 Pond_01-Feb-20+(Predict+ChargeOff)-Copy1.ipynb
-rw-r--r-- 1 root root    128831 Feb 25  2020 Pond_01-Feb-20+(Predict+ChargeOff).ipynb
-rw-r--r-- 1 root root     19082 Feb  8  2020 Pond_07-Sep-19 (Homework).ipynb
-rw-r--r-- 1 root root      9710 Feb  8  2020 Pond_07-Sep-19 (Time Table).ipynb
-rw-r--r-- 1 root root    115459 Dec  8  2019 Pond_08-Dec-19 (Arrival Delay)

In [10]:
Lending_Club_df = spark.read.format('csv').\
option('header','true').option('mode','DROPMALFORMED')\
.load('/root/notebook/data/LoanStats_web.csv')

In [11]:
#นับจำนวนข้อมูล (Row) เป็นการตรวจสอบจำนวนข้อมูลทั้งหมด

In [12]:
Lending_Club_df.count()

1432439

In [13]:
#นับจำนวน Field (Columns) เป็นการตรวจสอบ Attributes

In [14]:
len(Lending_Club_df.columns)

144

In [15]:
#ดู Type ของ่ข้อมูล (String / Double / etc.) 

In [16]:
Lending_Club_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

In [17]:
#ดูกลุ่มข้อมูลของแต่ละตัวแปรและนับจำนวนในแต่ละกลุ่ม

In [18]:
Lending_Club_df.select(['purpose']).groupBy('purpose').count().show()

+------------------+------+
|           purpose| count|
+------------------+------+
|           wedding|     7|
|             other| 93997|
|    small_business| 15032|
|debt_consolidation|791531|
|       credit_card|335091|
|            moving|  9733|
|          vacation| 10579|
|  renewable_energy|   910|
|             house|  9893|
|               car| 15288|
|    major_purchase| 32649|
|           medical| 18807|
|  home_improvement| 98922|
+------------------+------+



In [19]:
Lending_Club_df.select(['home_ownership']).groupBy('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|           OWN|170888|
|          RENT|556960|
|      MORTGAGE|701646|
|           ANY|  2940|
|          NONE|     5|
+--------------+------+



In [20]:
Lending_Club_df.select(['grade']).groupBy('grade').count().show()

+-----+------+
|grade| count|
+-----+------+
|    F| 19480|
|    E| 68353|
|    B|424110|
|    D|191136|
|    C|421101|
|    A|302988|
|    G|  5271|
+-----+------+



In [21]:
Lending_Club_df.select(['emp_title']).groupBy('emp_title').count().show()

+--------------------+-----+
|           emp_title|count|
+--------------------+-----+
|           Physician| 1337|
|       Construction |   84|
|Maintenance Fligh...|    1|
|     Medical billing|   26|
|      sale associate|    7|
| Field Sales Manager|   24|
|Learning Behavior...|    1|
|          lieutenant|   26|
|Bookkeeper/office...|    1|
|      Histology Tech|   12|
|                 CSR|  694|
|      SUPERINTENDENT|   52|
|      Lead Therapist|   12|
|           Scientist|  317|
|       Swing Manager|    2|
|      CLERK TYPIST 3|    1|
|Vp financial cent...|    1|
|           machinist|  429|
|Branch Regional R...|    1|
|           hygienist|   15|
+--------------------+-----+
only showing top 20 rows



In [22]:
Lending_Club_df.select(['emp_length']).groupBy('emp_length').count().show()

+----------+------+
|emp_length| count|
+----------+------+
|   9 years| 46393|
|   5 years| 88686|
|    1 year| 95926|
|       n/a|108470|
|   2 years|130250|
|   7 years| 48965|
|   8 years| 51222|
|   4 years| 86767|
|   6 years| 62261|
|   3 years|116802|
| 10+ years|478304|
|  < 1 year|118393|
+----------+------+



In [23]:
Lending_Club_df.select(['loan_status']).groupBy('loan_status').count().show()

+------------------+------+
|       loan_status| count|
+------------------+------+
|        Fully Paid|583755|
|           Default|  1419|
|   In Grace Period|  6136|
|       Charged Off|152998|
|Late (31-120 days)| 19455|
|           Current|663690|
| Late (16-30 days)|  4986|
+------------------+------+



In [24]:
Lending_Club_df.select(['term']).groupBy('term').count().show()

+----------+-------+
|      term|  count|
+----------+-------+
| 36 months|1036234|
| 60 months| 396205|
+----------+-------+



In [25]:
Lending_Club_df.select(['pub_rec']).groupBy('pub_rec').count().show()

+-------+-------+
|pub_rec|  count|
+-------+-------+
|      7|    302|
|     15|      4|
|     11|     39|
|      3|   6827|
|      8|    111|
|     28|      9|
|     22|      1|
|     16|     14|
|     52|      1|
|      0|1196699|
|     47|      1|
|      5|   1300|
|     31|      1|
|     61|      5|
|     17|      2|
|     46|      1|
|      6|    629|
|     19|     16|
|     25|      1|
|      9|     99|
+-------+-------+
only showing top 20 rows



In [26]:
Lending_Club_df.select(['inq_last_6mths']).groupBy('inq_last_6mths').count().show()

+--------------+------+
|inq_last_6mths| count|
+--------------+------+
|             3| 31289|
|             0|925659|
|          null|     1|
|             5|  2400|
|             1|356887|
|             4|  7274|
|             2|108929|
+--------------+------+



In [27]:
Lending_Club_df.select(['settlement_status']).groupBy('settlement_status').count().show()

+-----------------+-------+
|settlement_status|  count|
+-----------------+-------+
|             null|1407028|
|         COMPLETE|   8606|
|           BROKEN|   4116|
|           ACTIVE|  12689|
+-----------------+-------+



### เลือกข้อมูลเฉพาะที่ Label เป็น "Fully Paid" และ "Charged Off"

In [28]:
from pyspark.sql.functions import *

In [29]:
Predict_ChargedOff_df = Lending_Club_df.filter((col('loan_status') == 'Fully Paid') | ((col('loan_status') =='Charged Off')))

In [30]:
Predict_ChargedOff_df.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|583755|
|Charged Off|152998|
+-----------+------+



In [31]:
Predict_ChargedOff_df.count()

736753

### คัดเลือกตัวแปร (Attributes) ที่เกี่ยวข้องกับ Business

In [32]:
raw_relatedAtt_df = Predict_ChargedOff_df

In [33]:
raw_business_df = raw_relatedAtt_df.select('annual_inc'\
                                           ,'bc_util'\
                                           ,'inq_fi'\
                                           ,'inq_last_12m'\
                                           ,'home_ownership'\
                                           ,'purpose'\
                                           ,'emp_length'\
                                           ,'revol_bal'\
                                           ,'dti'\
                                           ,'delinq_2yrs'\
                                           ,'pub_rec_bankruptcies'\
                                           ,'pub_rec'\
                                           ,'open_rv_24m'\
                                           ,'mort_acc'\
                                           ,'num_actv_bc_tl'\
                                           ,'num_actv_rev_tl'\
                                           ,'num_il_tl'\
                                           ,'num_tl_90g_dpd_24m'\
                                           ,'int_rate'\
                                           ,'inq_last_6mths'\
                                           ,'term'\
                                           ,'installment'\
                                           ,'total_rev_hi_lim'\
                                           ,'total_bal_il'\
                                           ,'total_bal_ex_mort'\
                                           ,'total_acc'\
                                           ,'tot_cur_bal'\
                                           ,'loan_amnt'\
                                           ,'loan_status'\
                                           ,'verification_status'\
                                           ,'collections_12_mths_ex_med'\
                                           ,'chargeoff_within_12_mths'\
                                           ,'il_util'\
                                           ,'last_pymnt_amnt'\
                                           #,'last_pymnt_d'\
                                           ,'out_prncp_inv'\
                                           ,'out_prncp'\
                                           ,'total_pymnt_inv'\
                                           ,'total_pymnt'\
                                           ,'grade')
                                           #,'sec_app_collections_12_mths_ex_med'\
                                           #,'sec_app_chargeoff_within_12_mths'\
                                           #,'settlement_term'\
                                           #,'settlement_amount'\
                                           #,'settlement_status'

In [34]:
raw_business_df.count()

736753

In [35]:
len(raw_business_df.columns)

39

In [36]:
! pip install pandas

You are using pip version 9.0.1, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### สร้าง Dataframe โดยใช้ Pandas และให้แสดงข้อมูลเบื้องต้น

In [37]:
import pandas as pd
pd.DataFrame(raw_business_df.describe().take(6),\
             columns=raw_business_df.describe().columns).transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
annual_inc,736753,79498.51232281375,78094.80304273874,0,99999.84
bc_util,727863,55.109972343696256,28.90306464396113,0,99.9
inq_fi,736697,1.058429720767154,1.5524238231939937,0,9
inq_last_12m,736696,2.226637581851944,2.492919247096462,0,9
home_ownership,736753,None,None,ANY,RENT
purpose,736753,None,None,car,wedding
emp_length,736753,None,None,1 year,n/a
revol_bal,736753,16101.816257280256,22819.366258659797,0,99991
dti,736281,18.711086881231356,13.79426027607045,-1,999


### 3) Data Preparation

### Cleansing ข้อมูล โดยการ Remove 'Null' ออก

In [38]:
no_null_df = raw_business_df.dropna(how='any')

In [39]:
no_null_df.count()

627714

### Remove ค่า Wedding ใน Field 'Purpose'

In [40]:
from pyspark.sql.functions import *

In [41]:
no_wedding_df = no_null_df.filter(col('purpose') != 'wedding')

In [42]:
no_wedding_df.select(['purpose']).groupBy('purpose').count().show()

+------------------+------+
|           purpose| count|
+------------------+------+
|             other| 41383|
|    small_business|  6552|
|debt_consolidation|360885|
|       credit_card|130378|
|            moving|  4877|
|          vacation|  5049|
|  renewable_energy|   445|
|             house|  3758|
|               car|  6838|
|    major_purchase| 14407|
|           medical|  8449|
|  home_improvement| 44688|
+------------------+------+



In [43]:
no_wedding_df.count()

627709

### นับจำนวนข้อมูลหลัง Cleansing ค่า 'Null'

In [44]:
fitmem_no_null_df = no_wedding_df.repartition(60)

In [45]:
cached_no_null_df = fitmem_no_null_df.cache()

In [46]:
cached_no_null_df.count()

627709

### ดู Type ของ่ข้อมูลหลัง Remove ค่า 'Null'

In [47]:
cached_no_null_df.printSchema()

root
 |-- annual_inc: string (nullable = true)
 |-- bc_util: string (nullable = true)
 |-- inq_fi: string (nullable = true)
 |-- inq_last_12m: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- pub_rec_bankruptcies: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- open_rv_24m: string (nullable = true)
 |-- mort_acc: string (nullable = true)
 |-- num_actv_bc_tl: string (nullable = true)
 |-- num_actv_rev_tl: string (nullable = true)
 |-- num_il_tl: string (nullable = true)
 |-- num_tl_90g_dpd_24m: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- term: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- total_rev_hi_lim: string (nullable = true)
 |-- total_bal_

### Remove '%' และ Extract Month

In [48]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove '%' 

In [49]:
def f_removepercent(origin):
    return origin.rstrip('%')

In [50]:
removepercent = udf(lambda x: f_removepercent(x),StringType())

### Remove 'Wedding'

In [51]:
#def f_wedding(origin):
    #return origin.rstrip('wedding')

In [52]:
#removewedding = udf(lambda x: f_wedding(x),StringType())

### Extract Month

In [53]:
def f_exrtractmonth(origin):
    return origin.split('-')[0]

In [54]:
exrtractmonth = udf(lambda x: f_exrtractmonth(x),StringType())

### Replace'n/a' ใน Field 'emp_length' ด้วย 'No employ'

In [55]:
def python_treatNA(origin):
    if origin == 'n/a':
        new = 'NotEmployed'
    else:
        new = origin
    return new

In [56]:
treatNA = udf(lambda x: python_treatNA(x),StringType())

### เปลี่ยนแปลง Type ของข้อมูล

In [57]:
from pyspark.sql.functions import col

In [58]:
crunched_df = cached_no_null_df.\
withColumn('emp_length',treatNA(cached_no_null_df['emp_length'])).\
withColumn('int_rate',removepercent(cached_no_null_df['int_rate']).cast(DoubleType())).\
withColumn('dti',cached_no_null_df['dti'].cast(DoubleType())).\
withColumn('revol_bal',cached_no_null_df['revol_bal'].cast(DoubleType())).\
withColumn('pub_rec',cached_no_null_df['pub_rec'].cast(DoubleType())).\
withColumn('total_bal_il',cached_no_null_df['total_bal_il'].cast(DoubleType())).\
withColumn('tot_cur_bal',cached_no_null_df['tot_cur_bal'].cast(DoubleType())).\
withColumn('total_acc',cached_no_null_df['total_acc'].cast(DoubleType())).\
withColumn('total_bal_ex_mort',cached_no_null_df['total_bal_ex_mort'].cast(DoubleType())).\
withColumn('total_rev_hi_lim',cached_no_null_df['total_rev_hi_lim'].cast(DoubleType())).\
withColumn('num_actv_rev_tl',cached_no_null_df['num_actv_rev_tl'].cast(DoubleType())).\
withColumn('num_actv_bc_tl',cached_no_null_df['num_actv_bc_tl'].cast(DoubleType())).\
withColumn('num_il_tl',cached_no_null_df['num_il_tl'].cast(DoubleType())).\
withColumn('pub_rec_bankruptcies',cached_no_null_df['pub_rec_bankruptcies'].cast(DoubleType())).\
withColumn('delinq_2yrs',cached_no_null_df['delinq_2yrs'].cast(DoubleType())).\
withColumn('open_rv_24m',cached_no_null_df['open_rv_24m'].cast(DoubleType())).\
withColumn('num_tl_90g_dpd_24m',cached_no_null_df['num_tl_90g_dpd_24m'].cast(DoubleType())).\
withColumn('inq_last_6mths',cached_no_null_df['inq_last_6mths'].cast(DoubleType())).\
withColumn('bc_util',cached_no_null_df['bc_util'].cast(DoubleType())).\
withColumn('mort_acc',cached_no_null_df['mort_acc'].cast(DoubleType())).\
withColumn('inq_fi',cached_no_null_df['inq_fi'].cast(DoubleType())).\
withColumn('last_pymnt_amnt',cached_no_null_df['last_pymnt_amnt'].cast(DoubleType())).\
withColumn('out_prncp_inv',cached_no_null_df['out_prncp_inv'].cast(DoubleType())).\
withColumn('out_prncp',cached_no_null_df['out_prncp'].cast(DoubleType())).\
withColumn('total_pymnt_inv',cached_no_null_df['total_pymnt_inv'].cast(DoubleType())).\
withColumn('total_pymnt',cached_no_null_df['total_pymnt'].cast(DoubleType())).\
withColumn('il_util',cached_no_null_df['il_util'].cast(DoubleType())).\
withColumn('chargeoff_within_12_mths',cached_no_null_df['chargeoff_within_12_mths'].cast(DoubleType())).\
withColumn('collections_12_mths_ex_med',cached_no_null_df['collections_12_mths_ex_med'].cast(DoubleType())).\
withColumn('loan_amnt',cached_no_null_df['loan_amnt'].cast(DoubleType())).\
withColumn('inq_last_12m',cached_no_null_df['inq_last_12m'].cast(DoubleType())).\
withColumn('installment',cached_no_null_df['installment'].cast(DoubleType())).\
withColumn('annual_inc',cached_no_null_df['annual_inc'].cast(DoubleType()))

In [59]:
crunched_df.printSchema()

root
 |-- annual_inc: double (nullable = true)
 |-- bc_util: double (nullable = true)
 |-- inq_fi: double (nullable = true)
 |-- inq_last_12m: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: double (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- open_rv_24m: double (nullable = true)
 |-- mort_acc: double (nullable = true)
 |-- num_actv_bc_tl: double (nullable = true)
 |-- num_actv_rev_tl: double (nullable = true)
 |-- num_il_tl: double (nullable = true)
 |-- num_tl_90g_dpd_24m: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- term: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- total_rev_hi_lim: double (nullable = true)
 |-- total_bal_

### เช็ค 'emp_length' หลัง Replace value

In [60]:
crunched_df.select(crunched_df['emp_length']).distinct().show()

+-----------+
| emp_length|
+-----------+
|    9 years|
|    5 years|
|     1 year|
|NotEmployed|
|    2 years|
|    7 years|
|    8 years|
|    4 years|
|    6 years|
|    3 years|
|  10+ years|
|   < 1 year|
+-----------+



In [61]:
crunched_df.select(crunched_df['purpose']).distinct().show()

+------------------+
|           purpose|
+------------------+
|             other|
|    small_business|
|debt_consolidation|
|       credit_card|
|            moving|
|          vacation|
|  renewable_energy|
|             house|
|               car|
|    major_purchase|
|           medical|
|  home_improvement|
+------------------+



### Normalization 'annual_inc'

In [62]:
from pyspark.sql.functions import *
max_annual_inc = crunched_df.select(max('annual_inc')).collect()[0][0]
min_annual_inc = crunched_df.select(min('annual_inc')).collect()[0][0]

In [63]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [64]:
n_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

### Normalization 'loan_amnt'

In [65]:
#max_loan_amt = crunched_df.select(max('loan_amnt')).collect()[0][0]
#min_loan_amt = crunched_df.select(min('loan_amnt')).collect()[0][0]

In [66]:
#def t_loan_amt(origin):
    #return ((origin-min_loan_amt)/(max_loan_amt-min_loan_amt))

In [67]:
#n_loan_amt = udf(lambda x: t_loan_amt(x),DoubleType())

### Normalization 'revol_bal'

In [68]:
max_revol_bal = crunched_df.select(max('revol_bal')).collect()[0][0]
min_revol_bal = crunched_df.select(min('revol_bal')).collect()[0][0]

In [69]:
def t_revol_bal(origin):
    return ((origin-min_revol_bal)/(max_revol_bal-min_revol_bal))

In [70]:
n_revol_bal = udf(lambda x: t_revol_bal(x),DoubleType())

### Normalization 'tot_cur_bal'

In [71]:
max_tot_cur_bal = crunched_df.select(max('tot_cur_bal')).collect()[0][0]
min_tot_cur_bal = crunched_df.select(min('tot_cur_bal')).collect()[0][0]

In [72]:
def t_tot_cur_bal(origin):
    return ((origin-min_tot_cur_bal)/(max_tot_cur_bal-min_tot_cur_bal))

In [73]:
n_tot_cur_bal = udf(lambda x: t_tot_cur_bal(x),DoubleType())

### Normalization 'total_rev_hi_lim'

In [74]:
max_total_rev_hi_lim = crunched_df.select(max('total_rev_hi_lim')).collect()[0][0]
min_total_rev_hi_lim = crunched_df.select(min('total_rev_hi_lim')).collect()[0][0]

In [75]:
def t_total_rev_hi_lim(origin):
    return ((origin-min_total_rev_hi_lim)/(max_total_rev_hi_lim-min_total_rev_hi_lim))

In [76]:
n_total_rev_hi_lim = udf(lambda x: t_total_rev_hi_lim(x),DoubleType())

### Normalization 'total_bal_ex_mort'

In [77]:
max_total_bal_ex_mort = crunched_df.select(max('total_bal_ex_mort')).collect()[0][0]
min_total_bal_ex_mort = crunched_df.select(min('total_bal_ex_mort')).collect()[0][0]

In [78]:
def t_total_bal_ex_mort(origin):
    return ((origin-min_total_bal_ex_mort)/(max_total_bal_ex_mort-min_total_bal_ex_mort))

In [79]:
n_total_bal_ex_mort = udf(lambda x: t_total_bal_ex_mort(x),DoubleType())

### Normalization 'total_bal_il'

In [80]:
max_total_bal_il = crunched_df.select(max('total_bal_il')).collect()[0][0]
min_total_bal_il = crunched_df.select(min('total_bal_il')).collect()[0][0]

In [81]:
def t_total_bal_il(origin):
    return ((origin-min_total_bal_il)/(max_total_bal_il-min_total_bal_il))

In [82]:
n_total_bal_il = udf(lambda x: t_total_bal_il(x),DoubleType())

### รวมตัวแปรที่ทำการ Normalized แล้ว

In [83]:
normalized_df = crunched_df.\
withColumn('annual_inc',n_annual_inc(crunched_df['annual_inc'])).\
withColumn('revol_bal',n_revol_bal(crunched_df['revol_bal'])).\
withColumn('tot_cur_bal',n_tot_cur_bal(crunched_df['tot_cur_bal'])).\
withColumn('total_rev_hi_lim',n_total_rev_hi_lim(crunched_df['total_rev_hi_lim'])).\
withColumn('total_bal_il',n_total_bal_il(crunched_df['total_bal_il'])).\
withColumn('total_bal_ex_mort',n_total_bal_ex_mort(crunched_df['total_bal_ex_mort']))

#withColumn('loan_amnt',n_loan_amt(crunched_df['loan_amnt'])).\

In [84]:
#ดูข้อมูลที่มีการทำ Normalization (ให้ลองเปลี่ยน orderby เป็น GroupBy)

In [85]:
normalized_df.select(['grade'\
                      ,'annual_inc'\
                      ,'revol_bal'\
                      ,'tot_cur_bal'\
                      ,'revol_bal'\
                      ,'total_bal_il'\
                      ,'total_rev_hi_lim']).orderBy(['grade']).show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|grade|          annual_inc|           revol_bal|         tot_cur_bal|           revol_bal|        total_bal_il|    total_rev_hi_lim|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    A|0.005742893408359239|0.004191086843994723|0.026179372974751938|0.004191086843994723| 0.09994269424113393|  0.0351493848857645|
|    A|0.003340315086656...|0.004729802766697...|0.002708901284331421|0.004729802766697...|0.006795447456422235| 0.01621407109246556|
|    A|0.008354391584123303|0.005709685725609922| 0.02314724007954436|0.005709685725609922|0.017692246313208614|0.013946368841771075|
|    A|0.011801569176131867|0.017764443167227792| 0.06389535964291722|0.017764443167227792| 0.04838346982742987|0.041498951187709056|
|    A|0.008563311438184429|0.002925587775982058| 0.0523365237

In [86]:
normalized_df.count()

627709

In [87]:
normalized_filtered_df = normalized_df\
#.filter(col('loan_status') == 'Fully Paid').\
#union(normalized_df.filter(col('loan_status') == 'Charged Off'))

In [88]:
normalized_filtered_df.count()

627709

In [89]:
#normalized_filtered_df.select('loan_status').distinct().show()

### สร้าง Dataframe โดยใช้ Pandas และให้แสดงข้อมูลหลัง Normalized

In [90]:
pd.DataFrame(normalized_df.describe().take(6),\
             columns=normalized_df.describe().columns).transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
annual_inc,627709,0.00846256614532977,0.008274208621998921,0.0,1.0
bc_util,627709,55.70084688924322,28.831685669621613,0.0,252.3
inq_fi,627709,1.1189500230202212,1.5859378645745748,0.0,48.0
inq_last_12m,627709,2.3041584555900902,2.5282249368970446,0.0,67.0
home_ownership,627709,None,None,ANY,RENT
purpose,627709,None,None,car,vacation
emp_length,627709,None,None,1 year,NotEmployed
revol_bal,627709,0.0142652576807312,0.019837633181117035,0.0,1.0
dti,627709,19.848822623221896,13.332388424437127,-1.0,999.0


### กำจัดค่า 'Null' จากการทำ Data Prep

In [91]:
data_no_missing_df = normalized_filtered_df.dropna(how='any')

In [92]:
data_no_missing_df.count()

627709

In [93]:
data_no_missing_df.printSchema()

root
 |-- annual_inc: double (nullable = true)
 |-- bc_util: double (nullable = true)
 |-- inq_fi: double (nullable = true)
 |-- inq_last_12m: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: double (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- open_rv_24m: double (nullable = true)
 |-- mort_acc: double (nullable = true)
 |-- num_actv_bc_tl: double (nullable = true)
 |-- num_actv_rev_tl: double (nullable = true)
 |-- num_il_tl: double (nullable = true)
 |-- num_tl_90g_dpd_24m: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- term: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- total_rev_hi_lim: double (nullable = true)
 |-- total_bal_

### 4) ML Modeling

In [94]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, \
OneHotEncoder, VectorIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, \
BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, \
NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [95]:
#เปลี่ยนข้อมูลที่ต้องการ (Field) จาก Categorical เป็น Numerical

In [96]:
labelIndexer = StringIndexer(inputCol='loan_status',outputCol='indexedLabel')

In [97]:
gradeIndexer = StringIndexer(inputCol='grade',outputCol='gradeIndexed')
gradeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='gradeIndexed',\
                                  outputCol='gradeVec')

In [98]:
homeIndexer = StringIndexer(inputCol='home_ownership',outputCol='homeIndexed')
homeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='homeIndexed',\
                                  outputCol='homeVec')

In [99]:
purposeIndexer = StringIndexer(inputCol='purpose',outputCol='purposeIndexed')
purposeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='purposeIndexed',\
                                  outputCol='purposeVec')

In [100]:
emp_lengthIndexer = StringIndexer(inputCol='emp_length',outputCol='emp_lengthIndexed')
emp_lengthOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='emp_lengthIndexed',\
                                  outputCol='emp_lengthVec')

In [101]:
verification_statusIndexer = StringIndexer(inputCol='verification_status',outputCol='verification_statusIndexed')
verification_statusOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='verification_statusIndexed',\
                                  outputCol='verification_statusVec')

### เลือก Feature ที่จะใช้ในการ Train

In [102]:
featureAssembler = VectorAssembler(inputCols=['annual_inc'\
                                              #,'bc_util'\
                                              #,'inq_fi'\
                                              #,'inq_last_12m'\
                                              #,'home_ownership'\
                                              #,'purpose'\
                                              #,'emp_length'\
                                              ,'revol_bal'\
                                              ,'dti'\
                                              ,'delinq_2yrs'\
                                              ,'pub_rec_bankruptcies'\
                                              ,'pub_rec'\
                                              #,'open_rv_24m'\
                                              ,'mort_acc'\
                                              ,'num_actv_bc_tl'\
                                              #,'num_actv_rev_tl'\
                                              #,'num_il_tl'\
                                              #,'num_tl_90g_dpd_24m'\
                                              #,'int_rate'\
                                              ,'inq_last_6mths'\
                                              #,'term'\
                                              ,'installment'\
                                              #,'total_rev_hi_lim'\
                                              ,'total_bal_il'\
                                              ,'total_bal_ex_mort'\
                                              ,'total_acc'\
                                              ,'tot_cur_bal'\
                                              ,'collections_12_mths_ex_med'\
                                              ,'chargeoff_within_12_mths'\
                                              ,'il_util'\
                                              ,'last_pymnt_amnt'\
                                              #,'last_pymnt_d'\
                                              ,'out_prncp_inv'\
                                              ,'out_prncp'\
                                              ,'total_pymnt_inv'\
                                              ,'total_pymnt'\
                                              ,'loan_amnt'\
                                              #,'loan_status'\
                                              #,'verification_status'\
                                              ,'gradeVec'\
                                              ,'homeVec'\
                                              ,'emp_lengthVec'\
                                              ,'purposeVec'\
                                              ,'verification_statusVec']\
                                   ,outputCol='***features')

# Modeling with DecisionTree

In [103]:
dt = DecisionTreeClassifier(featuresCol='***features',labelCol='indexedLabel')

In [104]:
#ทำ Sequencing ลำดับของการทำ Modeling

In [105]:
pipeline_dt = Pipeline().setStages([gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,\
                                    dt])

### แบ่งข้อมูลเป็น Training และ Testing

In [106]:
#แบ่งข้อมูลเป็น Training และ Testing (ุ60% - 40%) โดย Random จากข้อมูลที่เหลืออยู่

In [107]:
training_dt, test_dt = normalized_df.randomSplit([0.6,0.4])

In [108]:
#ดูการการกระจุกตัวของ Label จากการ Random เพื่อดู Imbalance Data

In [109]:
training_dt.groupBy('loan_status').count().orderBy(['loan_status']).show()

+-----------+------+
|loan_status| count|
+-----------+------+
|Charged Off| 78737|
| Fully Paid|297427|
+-----------+------+



In [110]:
training_dt.groupBy('grade').count().show()

+-----+------+
|grade| count|
+-----+------+
|    F|  7542|
|    E| 22113|
|    B|111724|
|    D| 52806|
|    C|113054|
|    A| 66690|
|    G|  2235|
+-----+------+



In [111]:
training_dt.count()

376164

In [112]:
#แบ่งข้อมูลเป็น Training และ Testing (ุ60%) โดย Random จาก Label ในแต่ละ Catergory 

In [113]:
#training, test = normalized_df\
#.filter(col('grade') == 'A').sample(False,0.4,42).\
#union(normalized_df.filter(col('grade') =='B').\
      #sample(False,0.4,42)).\
#union(normalized_df.filter(col('grade') =='C').\
      #ample(False,0.4,42)).\
#union(normalized_df.filter(col('grade') =='D').\
      #sample(False,0.4,42)).\
#union(normalized_df.filter(col('grade') =='E')).\
#union(normalized_df.filter(col('grade') =='F')).\
#union(normalized_df.filter(col('grade') =='G')).\
#randomSplit([0.4,0.2], seed = 12)

In [114]:
start_time_dt = t.time()

In [115]:
model_dt = pipeline_dt.fit(training_dt)

In [116]:
duration_dt = t.time() - start_time_dt

In [117]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_dt))

ใช้เวลา Distributed Training 80.90866780281067 วินาที


In [118]:
spark_fullLabel_func = udf(lambda x: "Fully Paid" if x == 0.0 else "Charged Off")

In [119]:
result_dt = model_dt.transform(test_dt.dropna()).withColumn('full_prediction',spark_fullLabel_func(col('prediction')))

In [120]:
#ดูการการกระจุกตัวของ Label จากการ Random เพื่อดู Imbalance Data (ข้อมูล Test)

In [121]:
result_dt.groupBy(['loan_status','indexedLabel']).count().show()

+-----------+------------+------+
|loan_status|indexedLabel| count|
+-----------+------------+------+
| Fully Paid|         0.0|199026|
|Charged Off|         1.0| 52519|
+-----------+------------+------+



In [122]:
result_dt.count()

251545

In [123]:
result_dt

DataFrame[annual_inc: double, bc_util: double, inq_fi: double, inq_last_12m: double, home_ownership: string, purpose: string, emp_length: string, revol_bal: double, dti: double, delinq_2yrs: double, pub_rec_bankruptcies: double, pub_rec: double, open_rv_24m: double, mort_acc: double, num_actv_bc_tl: double, num_actv_rev_tl: double, num_il_tl: double, num_tl_90g_dpd_24m: double, int_rate: double, inq_last_6mths: double, term: string, installment: double, total_rev_hi_lim: double, total_bal_il: double, total_bal_ex_mort: double, total_acc: double, tot_cur_bal: double, loan_amnt: double, loan_status: string, verification_status: string, collections_12_mths_ex_med: double, chargeoff_within_12_mths: double, il_util: double, last_pymnt_amnt: double, out_prncp_inv: double, out_prncp: double, total_pymnt_inv: double, total_pymnt: double, grade: string, gradeIndexed: double, gradeVec: vector, homeIndexed: double, homeVec: vector, emp_lengthIndexed: double, emp_lengthVec: vector, purposeIndexed:

In [124]:
num_result_dt = result_dt.count()

# การประเมิน Charge-Off [Aekanun]

In [125]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [126]:
num_Actual_ChargeOff = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').count()

In [127]:
num_Actual_ChargeOff

52519

In [128]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [129]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [130]:
num_False_FullyPaid = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [131]:
(num_False_FullyPaid/num_Actual_ChargeOff)*100

18.18199127934652

In [132]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|  

In [133]:
num_True_ChargeOff = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [134]:
(num_True_ChargeOff/num_Actual_ChargeOff)*100

81.81800872065348

# ------------- สิ้นสุดการประเมิน ---------------------------

# การประเมิน Fully Paid [Aekanun]

In [135]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [136]:
num_Actual_FullyPaid = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').count()

In [137]:
num_Actual_FullyPaid

199026

In [138]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [139]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|  

In [140]:
num_False_ChargedOff = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [141]:
(num_False_ChargedOff/num_Actual_FullyPaid)*100

0.5315888376393034

In [142]:
result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [143]:
num_True_FullyPaid = result_dt.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [144]:
(num_True_FullyPaid/num_Actual_FullyPaid)*100

99.4684111623607

# ------------- สิ้นสุดการประเมิน ---------------------------

In [145]:
num_Actual_FullyPaid + num_Actual_ChargeOff

251545

In [146]:
(num_True_FullyPaid+num_True_ChargeOff)/(num_Actual_FullyPaid + num_Actual_ChargeOff)

0.9578325945655847

In [147]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [148]:
evaluator_DT = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_DT.evaluate(result_dt)*100

95.78325945655847

# Modeling with RandomForest

In [149]:
RF = RandomForestClassifier(featuresCol='***features',labelCol='indexedLabel')

In [150]:
pipeline_RF = Pipeline().setStages([gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,\
                                    RF])

In [151]:
#training_RF, test_RF = normalized_df.randomSplit([0.6,0.4])

In [152]:
training_RF = training_dt

In [153]:
test_RF = test_dt

In [154]:
training_RF.groupBy('loan_status').count().orderBy(['loan_status']).show()

+-----------+------+
|loan_status| count|
+-----------+------+
|Charged Off| 78737|
| Fully Paid|297427|
+-----------+------+



In [155]:
training_RF.count()

376164

In [156]:
start_time_rf = t.time()

In [157]:
model_RF = pipeline_RF.fit(training_RF)

In [158]:
duration_rf = t.time() - start_time_rf

In [159]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_rf))

ใช้เวลา Distributed Training 71.83115410804749 วินาที


In [160]:
result_RF = model_RF.transform(test_RF.dropna()).withColumn('full_prediction',spark_fullLabel_func(col('prediction')))

In [161]:
result_RF.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|199026|
|Charged Off| 52519|
+-----------+------+



In [162]:
result_RF.count()

251545

In [163]:
result_RF

DataFrame[annual_inc: double, bc_util: double, inq_fi: double, inq_last_12m: double, home_ownership: string, purpose: string, emp_length: string, revol_bal: double, dti: double, delinq_2yrs: double, pub_rec_bankruptcies: double, pub_rec: double, open_rv_24m: double, mort_acc: double, num_actv_bc_tl: double, num_actv_rev_tl: double, num_il_tl: double, num_tl_90g_dpd_24m: double, int_rate: double, inq_last_6mths: double, term: string, installment: double, total_rev_hi_lim: double, total_bal_il: double, total_bal_ex_mort: double, total_acc: double, tot_cur_bal: double, loan_amnt: double, loan_status: string, verification_status: string, collections_12_mths_ex_med: double, chargeoff_within_12_mths: double, il_util: double, last_pymnt_amnt: double, out_prncp_inv: double, out_prncp: double, total_pymnt_inv: double, total_pymnt: double, grade: string, gradeIndexed: double, gradeVec: vector, homeIndexed: double, homeVec: vector, emp_lengthIndexed: double, emp_lengthVec: vector, purposeIndexed:

# การประเมิน Charge-Off [Aekanun]

In [164]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [165]:
num_Actual_ChargeOff = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').count()

In [166]:
num_Actual_ChargeOff

52519

In [167]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [168]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [169]:
num_False_FullyPaid = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [170]:
num_False_FullyPaid

27203

In [171]:
(num_False_FullyPaid/num_Actual_ChargeOff)*100

51.79649269788077

In [172]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|  

In [173]:
num_True_ChargeOff = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [174]:
(num_True_ChargeOff/num_Actual_ChargeOff)*100

48.20350730211923

# ------------- สิ้นสุดการประเมิน ---------------------------

# การประเมิน Fully Paid [Aekanun]

In [175]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [176]:
num_Actual_FullyPaid = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').count()

In [177]:
num_Actual_FullyPaid

199026

In [178]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [179]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|  

In [180]:
num_False_ChargedOff = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [181]:
(num_False_ChargedOff/num_Actual_FullyPaid)*100

0.05778139539557646

In [182]:
result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [183]:
num_True_FullyPaid = result_RF.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [184]:
(num_True_FullyPaid/num_Actual_FullyPaid)*100

99.94221860460442

# ------------- สิ้นสุดการประเมิน ---------------------------

In [185]:
num_Actual_FullyPaid + num_Actual_ChargeOff

251545

In [186]:
(num_True_FullyPaid+num_True_ChargeOff)/(num_Actual_FullyPaid + num_Actual_ChargeOff)

0.89139915323302

In [187]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [188]:
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_RF.evaluate(result_RF)*100

89.139915323302

# Modeling with Multi layer Perceptron (ANN)

In [189]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [190]:
pipeline_mlp = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler])

In [191]:
#training_mlp, test_mlp = normalized_df.randomSplit([0.6,0.4])

In [192]:
training_mlp = training_dt

In [193]:
test_mlp = test_dt

In [194]:
train_df_features = pipeline_mlp.fit(training_mlp).transform(training_mlp)

In [195]:
layers = [train_df_features.schema["***features"].metadata["ml_attr"]["num_attrs"],20 ,10, 2]

In [196]:
layers

[62, 20, 10, 2]

### Implementation of MLP

In [197]:
clf = MultilayerPerceptronClassifier(labelCol='indexedLabel'\
                                    ,featuresCol='***features'\
                                    ,layers = layers)

In [198]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [199]:
pipeline_mlp_test = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    verification_statusIndexer,verification_statusOneHotEncoder,\
                                    labelIndexer,\
                                    featureAssembler,
                                    clf])

In [200]:
#training_mlp_test, test_mlp_test = normalized_df.randomSplit([0.6,0.4])

In [201]:
training_mlp.groupBy('loan_status').count().orderBy(['loan_status']).show()

+-----------+------+
|loan_status| count|
+-----------+------+
|Charged Off| 78737|
| Fully Paid|297427|
+-----------+------+



In [202]:
training_mlp.count()

376164

In [203]:
start_time_mlp = t.time()

In [204]:
model_mlp = pipeline_mlp_test.fit(training_mlp)

In [205]:
duration_mlp = t.time() - start_time_mlp

In [206]:
print('ใช้เวลา Distributed Training {0} วินาที'.format(duration_mlp))

ใช้เวลา Distributed Training 255.3360812664032 วินาที


In [207]:
result_mlp = model_mlp.transform(test_mlp.dropna())\
.withColumn('full_prediction',spark_fullLabel_func(col('prediction')))

In [208]:
result_mlp.groupBy('loan_status').count().show()

+-----------+------+
|loan_status| count|
+-----------+------+
| Fully Paid|199026|
|Charged Off| 52519|
+-----------+------+



In [209]:
result_mlp.count()

251545

In [210]:
result_mlp

DataFrame[annual_inc: double, bc_util: double, inq_fi: double, inq_last_12m: double, home_ownership: string, purpose: string, emp_length: string, revol_bal: double, dti: double, delinq_2yrs: double, pub_rec_bankruptcies: double, pub_rec: double, open_rv_24m: double, mort_acc: double, num_actv_bc_tl: double, num_actv_rev_tl: double, num_il_tl: double, num_tl_90g_dpd_24m: double, int_rate: double, inq_last_6mths: double, term: string, installment: double, total_rev_hi_lim: double, total_bal_il: double, total_bal_ex_mort: double, total_acc: double, tot_cur_bal: double, loan_amnt: double, loan_status: string, verification_status: string, collections_12_mths_ex_med: double, chargeoff_within_12_mths: double, il_util: double, last_pymnt_amnt: double, out_prncp_inv: double, out_prncp: double, total_pymnt_inv: double, total_pymnt: double, grade: string, gradeIndexed: double, gradeVec: vector, homeIndexed: double, homeVec: vector, emp_lengthIndexed: double, emp_lengthVec: vector, purposeIndexed:

# การประเมิน Charge-Off [Aekanun]

In [211]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [212]:
num_Actual_ChargeOff = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Charged Off').count()

In [213]:
num_Actual_ChargeOff

52519

In [214]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Positive = Charged Off

In [215]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|       0.0|     Fully Paid|
|Charged Off|         1.0|  

In [216]:
num_False_FullyPaid = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [217]:
num_False_FullyPaid

1656

In [218]:
(num_False_FullyPaid/num_Actual_ChargeOff)*100

3.1531445762485957

In [219]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|       1.0|    Charged Off|
|Charged Off|         1.0|  

In [220]:
num_True_ChargeOff = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Charged Off').count()

In [221]:
(num_True_ChargeOff/num_Actual_ChargeOff)*100

96.84685542375139

# ------------- สิ้นสุดการประเมิน ---------------------------

# การประเมิน Fully Paid [Aekanun]

In [222]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [223]:
num_Actual_FullyPaid = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter(col('loan_status') == 'Fully Paid').count()

In [224]:
num_Actual_FullyPaid

199026

In [225]:
#ตรวจสอบความถูกต้องจากการ Predict ตรงกับข้อมูลที่เป็น IndexLabel หรือไม่ (เนื่องจากมีการทำ indexer ค่าที่ได้จึงเป็น Numerical)

### Negative = Fully Paid

In [226]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
| Fully Paid|         0.0|       1.0|    Charged Off|
+-----------+------------+--

In [227]:
num_False_ChargedOff = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') != col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [228]:
(num_False_ChargedOff/num_Actual_FullyPaid)*100

0.007536703747249103

In [229]:
result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').show()

+-----------+------------+----------+---------------+
|loan_status|indexedLabel|prediction|full_prediction|
+-----------+------------+----------+---------------+
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|       0.0|     Fully Paid|
| Fully Paid|         0.0|  

In [230]:
num_True_FullyPaid = result_mlp.select('loan_status','indexedLabel','prediction','full_prediction').\
filter((col('indexedLabel') == col('prediction'))).filter(col('loan_status') == 'Fully Paid').count()

In [231]:
(num_True_FullyPaid/num_Actual_FullyPaid)*100

99.99246329625275

# ------------- สิ้นสุดการประเมิน ---------------------------

In [232]:
num_Actual_FullyPaid + num_Actual_ChargeOff

251545

In [233]:
(num_True_FullyPaid+num_True_ChargeOff)/(num_Actual_FullyPaid + num_Actual_ChargeOff)

0.9933570534099266

In [234]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [235]:
evaluator_RF = MulticlassClassificationEvaluator(predictionCol="prediction",\
                                              labelCol='indexedLabel', metricName='accuracy')
evaluator_RF.evaluate(result_mlp)*100

99.33570534099266

In [236]:
#model_mlp.save('./modelMLP_bestloanpayment')

In [237]:
! df -h

Filesystem                      Size  Used Avail Use% Mounted on
/dev/mapper/set1--vm1--vg-root   12G   11G  662M  95% /
none                            4.0K     0  4.0K   0% /sys/fs/cgroup
udev                            3.9G  4.0K  3.9G   1% /dev
tmpfs                           798M  840K  797M   1% /run
none                            5.0M     0  5.0M   0% /run/lock
none                            3.9G     0  3.9G   0% /run/shm
none                            100M   12K  100M   1% /run/user
/dev/sda1                       236M   37M  188M  17% /boot


In [238]:
! ls -l modelMLP_bestloanpayment

total 8
drwxr-xr-x  2 root root 4096 Oct 18 19:36 metadata
drwxr-xr-x 15 root root 4096 Oct 18 19:36 stages
